Steps
* calculate size of electrode areas 4680 vs 280Ah vs 500Ah 
* determine number of windings / sheets 
* calculate number of current collector tabs 
* cell volume utilization ratio (electode-seperator-current_collector volume / cell can volume) --> why?
* resistance / impedance to calculate Joule Heating IR² (only valid for external short circuit, not internal)


In [1]:
import sys
sys.path.insert(0, '/home/matthias/Documents/GitHub/Cell_model_energy_costs')
import cell_model as Cellmodel
import pandas as pd

## Making the cells

In [2]:
# Active materials
NMC_cathode = Cellmodel.Activematerial_cathode("NMC811" , 195, 3.86)
LFP_cathode = Cellmodel.Activematerial_cathode("LFP", 155, 3.3)
Gr = Cellmodel.Activematerial_anode("Gr", 344 , 0.17)

# Currentcollectors 
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)

# Electrolyte 
LP40_standard = Cellmodel.Electrolyte_liquid("LiPF6" , "EC:DMC",
                                      1.1, # concentration mol / L
                                      151.9 # MolarMass LiPF6
                                      )

# Separator
Separator_standard = Cellmodel.Separator("PP+Al" , 12e-4 , 0.44 , 1.18)

# Electrods 
positive_NMC = Cellmodel.Electrodecomposition_cathode_opt1(NMC_cathode.name,
                                                       5.4, # areal capacity
                                                       0.95, # active frac
                                                       3.4, # density
                                                       NMC_cathode
                                                      ) # NMC = 3.4 / LFP = 2.5
positive_LFP = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name, 
                                                        3, 
                                                        0.92, 
                                                        2.5, 
                                                        LFP_cathode
                                                        ) # NMC = 3.4 / LFP = 2.5
negative_NMC = Cellmodel.Electrodecomposition_anode_opt1( Gr.name , positive_NMC.areal_cap*1.1 , 0.965 , 1.6, Gr)
negative_LFP = Cellmodel.Electrodecomposition_anode_opt1( Gr.name , positive_LFP.areal_cap*1.1 , 0.965 , 1.6, Gr)

* The NMC cathode needs to be quite thick, so 5 mAh / cm2 gives about 80 micrometer

### Cylindrical

In [3]:
NMC_Gr_cyl = Cellmodel.Cylindrical("NMC-Gr_Cyl", 
                               positive_NMC, 
                               negative_NMC, 
                               Separator_standard , 
                               LP40_standard,
                               1.7, # electrolyte content mL/mAh
                               4.6, # diameter
                               8.0, # height 
                               0.06, # can thickness
                               7.9, # can density
                               0.3, # void diameter
                               0.6, # head space
                               4, # extra mass
                               0.94, # LLI
                               NMC_cathode, 
                               Gr, 
                               Al, 
                               Cu)

In [4]:
cylindrical_surf_area = 2 * 3.14 * (NMC_Gr_cyl.diameter/2) * NMC_Gr_cyl.height \
                        + 2 * (3.14 * (NMC_Gr_cyl.diameter/2)**2)
cylindrical_surf_area

148.77319999999997

In [5]:
NMC_Gr_cyl.jr_area

2488.308195845172

In [6]:
NMC_Gr_cyl.turns

46.37715059996704

In [7]:
positive_NMC.thickness_el_c

0.008573469873779471

In [8]:
NMC_Gr_cyl.energy

93.21421472757248

In [9]:
NMC_Gr_cyl.capacity

25.261304804220185

In [10]:
2376/NMC_Gr_cyl.jr_area

0.9548656408266879

Seems to fit well to the Tesla 4680 NMC811 cell https://insideevs.com/news/598656/tesla-4680-battery-cell-specs/ 

### Prismatic 280 Ah

In [11]:
LFP_Gr_prismatic = Cellmodel.Prismatic("LFP-Gr_Pris", 
                                       positive_LFP, 
                                       negative_LFP, 
                                       Separator_standard, 
                                       LP40_standard, 
                                       1.7, # electrolyte ratio mL / mAh
                                       17, # height
                                       20, # width 
                                       7, # depth
                                       0.3, # can thickness in cm
                                       7.9, # can density
                                       2, # terminal clearance
                                       1, # nrolls
                                       40, # extramass
                                       0.94, # LLI
                                       LFP_cathode, 
                                       Gr, 
                                       Al, 
                                       Cu)

# CATL dimensions 173.9*71.7*207.2

Assuming only 1 jelly roll in the prismatic can

In [12]:
prismatic_surf_area = 2 * (LFP_Gr_prismatic.length * LFP_Gr_prismatic.width) \
                        + 2 * (LFP_Gr_prismatic.length * LFP_Gr_prismatic.height) \
                        + 2 * (LFP_Gr_prismatic.width * LFP_Gr_prismatic. height) 
prismatic_surf_area

69938.81162105073

In [13]:
LFP_Gr_prismatic.jr_area

52016.42830156434

In [14]:
LFP_Gr_prismatic.turns

93.80392686506475

In [15]:
positive_LFP.thickness_el_c

0.008415147265077139

In [16]:
LFP_Gr_prismatic.energy

918.2564120931755

In [17]:
LFP_Gr_prismatic.capacity

293.37265562082285

In [18]:
280/LFP_Gr_prismatic.capacity

0.9544175117734671

Seems to fit to the CATL 280 Ah cell 
* https://www.evlithium.com/CATL-Battery-Cell/catl-lifepo4-battery-280ah.html 
* https://www.catl.com/en/uploads/1/file/public/202205/20220512225901_d0k23qtykm.pdf

### Prismatic 560 Ah

In [19]:
LFP_Gr_prismatic_560 = Cellmodel.Prismatic("LFP-Gr_Pris", 
                                       positive_LFP, 
                                       negative_LFP, 
                                       Separator_standard, 
                                       LP40_standard, 
                                       1.7, # electrolyte ratio mL / mAh
                                       17, # height
                                       20*2, # width 
                                       7, # depth
                                       0.5, # can thickness
                                       7.9, # can density
                                       2, # terminal clearance
                                       1, # nrolls
                                       40, # extramass
                                       0.94, # LLI
                                       LFP_cathode, 
                                       Gr, 
                                       Al, 
                                       Cu)

Assuming only 1 jelly roll in the prismatic can

In [20]:
prismatic_surf_area_560 = 2 * (LFP_Gr_prismatic_560.length * LFP_Gr_prismatic_560.width) \
                        + 2 * (LFP_Gr_prismatic_560.length * LFP_Gr_prismatic_560.height) \
                        + 2 * (LFP_Gr_prismatic_560.width * LFP_Gr_prismatic_560. height) 
prismatic_surf_area_560

95041.0821223303

In [21]:
LFP_Gr_prismatic_560.jr_area

101097.77817399082

In [22]:
LFP_Gr_prismatic_560.turns

87.89664004064325

In [23]:
LFP_Gr_prismatic_560.energy

1784.6992976610945

In [24]:
LFP_Gr_prismatic_560.capacity

570.1914689013082

Seems to be in an reasonable range compared to the reported values for the EVE cell
* https://www.takomabattery.com/eve-releases-560ah-large-capacity-energy-storage-battery-lf560k/


## Combining info

In [25]:
energy_pack = 100000 # Wh
df_all = pd.DataFrame(index = ['Jelly Roll Area in cm2', 
                                'Turns in #', 
                                'Capacity in Ah', 
                                'Energy in Wh', 
                                'Cells@100kWh in #', 
                                'Outer Surface Area in cm2',
                                'Jelly Rol Area per Energy in cm2/Wh',
                                'Turns per Energy in #/mWh',
                                'Outer Surface Area per Energy in cm2/Wh'])
df_all['Cylindric 4680'] = [NMC_Gr_cyl.jr_area, 
                             NMC_Gr_cyl.turns, 
                             NMC_Gr_cyl.capacity,
                             NMC_Gr_cyl.energy,
                             energy_pack/NMC_Gr_cyl.energy, 
                            cylindrical_surf_area, 
                            NMC_Gr_cyl.jr_area/NMC_Gr_cyl.energy,
                            NMC_Gr_cyl.turns/(NMC_Gr_cyl.energy/1000),
                           cylindrical_surf_area/NMC_Gr_cyl.jr_area]
df_all['Prismatic 280Ah'] = [LFP_Gr_prismatic.jr_area,
                            LFP_Gr_prismatic.turns,
                            LFP_Gr_prismatic.capacity,
                            LFP_Gr_prismatic.energy,
                            energy_pack/LFP_Gr_prismatic.energy,
                            prismatic_surf_area,
                            LFP_Gr_prismatic.jr_area/LFP_Gr_prismatic.energy,
                            LFP_Gr_prismatic.turns/(LFP_Gr_prismatic.energy/1000),
                            prismatic_surf_area/LFP_Gr_prismatic.energy]
df_all['Prismatic 560Ah'] = [LFP_Gr_prismatic_560.jr_area,
                            LFP_Gr_prismatic_560.turns,
                            LFP_Gr_prismatic_560.capacity,
                            LFP_Gr_prismatic_560.energy,
                            energy_pack/LFP_Gr_prismatic_560.energy,
                            prismatic_surf_area_560,
                            LFP_Gr_prismatic_560.jr_area/LFP_Gr_prismatic_560.energy,
                            LFP_Gr_prismatic_560.turns/(LFP_Gr_prismatic_560.energy/1000),
                            prismatic_surf_area_560/LFP_Gr_prismatic_560.energy]
df_all.round(decimals=1)

,Cylindric 4680,Prismatic 280Ah,Prismatic 560Ah
Jelly Roll Area in cm2,2488.3,52016.4,101097.8
Turns in #,46.4,93.8,87.9
Capacity in Ah,25.3,293.4,570.2
Energy in Wh,93.2,918.3,1784.7
Cells@100kWh in #,1072.8,108.9,56.0
Outer Surface Area in cm2,148.8,69938.8,95041.1
Jelly Rol Area per Energy in cm2/Wh,26.7,56.6,56.6
Turns per Energy in #/mWh,497.5,102.2,49.3
Outer Surface Area per Energy in cm2/Wh,0.1,76.2,53.3


## Conclusions

What really creates the risks from a manufacturing viewpoint: 
* large electrode sheets -> it becomes more challenging to produce highly uniform and defect free large areas 
* number of stacks / turns in one cell -> the processes are performed at high speed in large quantities, but the precision of each step needs to nearly perfect, the more proceses performed, the higher the chances one of these is less perfect 

What really creates the risks from a system viewpoint: 
* amount of cells per system -> chances are larger, that there is one faulty cell in one large battery pack 
* outer surface area per energy -> if the ratio of outer surface area to energy is large, there is a big surface available which can be used to cool the cell, with a small ratio, only low cooling power can be directed within the cell 
* energy content per cell -> small cells could potentially go into a thermal runaway, but a thermal propagation can still be avoided, since the shielding can prevent other cells to be come affected. If the cell contains too much energy, thermal shielding might not be capable to protect the other cells from going into thermal runaway 


What do we see for these different cell classes: 
* the 280 Ah cell format shows highest ratio of outer surface area to energy and so provides the largest area for potential cooling and heat convection 
* the 280 Ah cell format shows roughly the same amount of turns or stackings in the housing in absolut terms, meaning it does not create less risks induces by the turns / stackings compared to the 560 Ah format 
* Leveraring these large cells reduces the amount of cells needed for a battery pack / rack significantly 